# Comparative Explanation work starts!

## Extracting 10 queries and appropriate features

In [1]:
import pandas as pd
import json

# Load the dataset
dataset_path = "fixed q2p_datapoints.json"
df = pd.read_json(dataset_path)

# Define the features to keep for each product
required_features = [
    'query',
    'product_title', 
    'base_price', 
    'final_price'
]

# Initialize a list to hold the processed data
processed_data = []

# Iterate over the first 10 rows (queries)
for index, row in df.iterrows():
    if index >= 10:  # Only process the first 10 queries
        break
    # Extract the query for the current row
    query = row['query']
    # Process each product column for the current query
    for col in ['product1', 'product2', 'product3']:
        product = row[col]
        # Initialize the processed product dictionary with the query
        processed_product = {'query': query}
        # Extract the required features for each product
        for feature in required_features:
            if feature in product:
                processed_product[feature] = product[feature]
        
        # Extract only the 'productRatingSummary' part of 'product_ugc_summary'
        
        processed_data.append(processed_product)

# Save the new smaller dataset as 10q2p.json
with open('c10.json', 'w') as outfile:
    json.dump(processed_data, outfile, indent=4)

print(f"Created new dataset 'c10.json' with {len(processed_data)} products.")


Created new dataset 'c10.json' with 30 products.


# Fixing price  values by removing .0

In [2]:
import json

# Load the data from the original JSON file
with open('c10.json', 'r') as file:
    data = json.load(file)

# Modify the prices to remove the decimal point
for item in data:
    item['base_price'] = int(item['base_price'])
    item['final_price'] = int(item['final_price'])

# Save the modified data to a new JSON file
with open('newc10.json', 'w') as file:
    json.dump(data, file, indent=4)

print("The file newc10.json has been created with updated prices.")


The file newc10.json has been created with updated prices.


# Appending product opinion summaries and creating finalc10.json

In [5]:
import json

# Load the data from newc10.json
with open('newc10.json', 'r') as file:
    new_c10_data = json.load(file)

# Load the data from Mistral_ProdOpSum_SpecSum_Eval.json
with open('Mistral_ProdOpSum_SpecSum_Eval.json', 'r') as file:
    mistral_data = json.load(file)

# Create a dictionary to easily map titles to product opinion summaries
mistral_dict = {item['product_title']: item.get('Product_Opinion_Summary', '') for item in mistral_data}

# Add the Product_Opinion_Summary to each product in newc10.json
for product in new_c10_data:
    title = product['product_title']
    product['Product_Opinion_Summary'] = mistral_dict.get(title, 'No summary available')

# Save the updated data to a new JSON file
with open('finalc10.json', 'w') as file:
    json.dump(new_c10_data, file, indent=4)

print("The file finalc10.json has been created with Product Opinion Summaries added.")


The file finalc10.json has been created with Product Opinion Summaries added.


# 1. Generating Comparative Explanation (BASE1)

In [1]:
import json
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Check if CUDA is available
if torch.cuda.is_available():
    # Specify CUDA device
    device = torch.device("cuda:1")
    print(f"Using CUDA device: {torch.cuda.get_device_name(device)}")
else:
    device = torch.device("cpu")
    print("CUDA is not available. Using CPU.")

# Load the Mistral 7B model
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
tokenizer.pad_token_id = tokenizer.eos_token_id
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2").to(device)

# Load the product data from finalc10.json
with open('finalc10.json', 'r') as f:
    product_data = json.load(f)

# System message and prompt
system_msg = "You are an expert in generating comparative explanations for product recommendations on an e-commerce platform. You carefully follow every instruction to answer faithfully, truthfully, and accurately in the specified format."
prompt = "### Instruction: Write a single comparative explanation using the user query, base price, final price, and Product Opinion Summary of all the recommended products for each query. The comparative explanation should provide a comparative analysis as to why product I was ranked above product II and why product II was ranked above product III. Write in a clear, engaging style for a general audience, and avoid overly technical language or jargon; aim for a conversational yet professional style while being fluent and coherent. The explanation should correctly and faithfully capture the key differences and reasons for the ranking of the products. Ensure that the prices are mentioned in Indian Rupees (Rs.) and not in any other currency. Strictly write the comparative explanation in the following format: {'Comparative Explanation': comparative explanation in one paragraph of 225-250 words.}"
# Initialize a dictionary to store the comparative explanations for each query

output_dict = {}

# Iterate over the products in the data
for i in range(0, len(product_data), 3):
    # Extract relevant fields for each product
    query = product_data[i]['query']
    product_titles = [product_data[i+j]['product_title'] for j in range(3)]
    base_prices = [product_data[i+j]['base_price'] for j in range(3)]
    final_prices = [product_data[i+j]['final_price'] for j in range(3)]
    opinion_summaries = [product_data[i+j]['Product_Opinion_Summary'] for j in range(3)]
    # Define a template for the input prompt
    input_template = f"""
    {system_msg}

    {prompt}

    User Query: {query}

    Product I:
    Title: {product_titles[0]}
    Base Price: Rs. {base_prices[0]}
    Final Price: Rs. {final_prices[0]}
    Opinion Summary: {opinion_summaries[0]}

    Product II:
    Title: {product_titles[1]}
    Base Price: Rs. {base_prices[1]}
    Final Price: Rs. {final_prices[1]}
    Opinion Summary: {opinion_summaries[1]}

    Product III:
    Title: {product_titles[2]}
    Base Price: Rs. {base_prices[2]}
    Final Price: Rs. {final_prices[2]}
    Opinion Summary: {opinion_summaries[2]}

    Instructions: 
    
    Please generate the comparative explanation following the above instructions in one single paragraph of 225-250 words.
    """

    # Generate the comparative explanation
    with torch.cuda.device(device):
        output = model.generate(
            max_length=4500,  # Increased max_length
            num_return_sequences=1,
            do_sample=True,
            top_k=25,
            top_p=0.95,
            num_beams=3,
            early_stopping=True,
            temperature=0.7,
            input_ids=tokenizer(input_template, return_tensors="pt").input_ids.to(device)
        )

    # Decode the comparative explanation
    comparative_explanation = tokenizer.decode(output[0], skip_special_tokens=True)

    # Add the query and comparative explanation to the dictionary
    output_dict[query] = {'Comparative Explanation': comparative_explanation}

    # Print the query and comparative explanation
    print("Query:", query)
    print("Comparative Explanation:", comparative_explanation)
    print("\n")  # Print newline for better readability between explanations


Using CUDA device: NVIDIA A100 80GB PCIe


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Query: Vivo 5G mobiles with curved display
Comparative Explanation: 
    You are an expert in generating comparative explanations for product recommendations on an e-commerce platform. You carefully follow every instruction to answer faithfully, truthfully, and accurately in the specified format.

    ### Instruction: Write a single comparative explanation using the user query, base price, final price, and Product Opinion Summary of all the recommended products for each query. The comparative explanation should provide a comparative analysis as to why product I was ranked above product II and why product II was ranked above product III. Write in a clear, engaging style for a general audience, and avoid overly technical language or jargon; aim for a conversational yet professional style while being fluent and coherent. The explanation should correctly and faithfully capture the key differences and reasons for the ranking of the products. Ensure that the prices are mentioned in Indian Ru

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Query: best 5G mobile with good camera and high performance under 25000
Comparative Explanation: 
    You are an expert in generating comparative explanations for product recommendations on an e-commerce platform. You carefully follow every instruction to answer faithfully, truthfully, and accurately in the specified format.

    ### Instruction: Write a single comparative explanation using the user query, base price, final price, and Product Opinion Summary of all the recommended products for each query. The comparative explanation should provide a comparative analysis as to why product I was ranked above product II and why product II was ranked above product III. Write in a clear, engaging style for a general audience, and avoid overly technical language or jargon; aim for a conversational yet professional style while being fluent and coherent. The explanation should correctly and faithfully capture the key differences and reasons for the ranking of the products. Ensure that the pric

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Query: buy dhop
Comparative Explanation: 
    You are an expert in generating comparative explanations for product recommendations on an e-commerce platform. You carefully follow every instruction to answer faithfully, truthfully, and accurately in the specified format.

    ### Instruction: Write a single comparative explanation using the user query, base price, final price, and Product Opinion Summary of all the recommended products for each query. The comparative explanation should provide a comparative analysis as to why product I was ranked above product II and why product II was ranked above product III. Write in a clear, engaging style for a general audience, and avoid overly technical language or jargon; aim for a conversational yet professional style while being fluent and coherent. The explanation should correctly and faithfully capture the key differences and reasons for the ranking of the products. Ensure that the prices are mentioned in Indian Rupees (Rs.) and not in any o

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Query: best 5G mobile under 15000 with good camera and great performance from Realme
Comparative Explanation: 
    You are an expert in generating comparative explanations for product recommendations on an e-commerce platform. You carefully follow every instruction to answer faithfully, truthfully, and accurately in the specified format.

    ### Instruction: Write a single comparative explanation using the user query, base price, final price, and Product Opinion Summary of all the recommended products for each query. The comparative explanation should provide a comparative analysis as to why product I was ranked above product II and why product II was ranked above product III. Write in a clear, engaging style for a general audience, and avoid overly technical language or jargon; aim for a conversational yet professional style while being fluent and coherent. The explanation should correctly and faithfully capture the key differences and reasons for the ranking of the products. Ensure 

# 1. Generating Comparative Explanation (BASE2)

In [2]:
import json
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Check if CUDA is available
if torch.cuda.is_available():
    # Specify CUDA device
    device = torch.device("cuda:1")
    print(f"Using CUDA device: {torch.cuda.get_device_name(device)}")
else:
    device = torch.device("cpu")
    print("CUDA is not available. Using CPU.")

# Load the Mistral 7B model
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
tokenizer.pad_token_id = tokenizer.eos_token_id
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2").to(device)

# Load the product data from finalc10.json
with open('finalc10.json', 'r') as f:
    product_data = json.load(f)

# System message and prompt
system_msg = "You are an expert in generating summaries that compare and justify the order of three recommended products on an e-commerce platform to help customers make purchase decisions. You carefully follow every instruction to answer faithfully, truthfully, and accurately in the specified format."
prompt = "### Instruction: Write a concise single paragraph summary that explains the order in which the three recommended products are presented for a given user query. Use information such as query, base price, final price, and Product Opinion Summary to justify the ranking. The comparative summary should provide a clear rationale for why product I is ranked above product II and why product II is ranked above product III, without providing individual explanations for each product. Write in a clear, engaging style for a general audience, avoiding overly technical language or jargon. Aim for a conversational yet professional tone while being fluent and coherent. The summary should correctly and faithfully capture the key differences and justify the reasons for the recommended order. Ensure that prices are mentioned in Indian Rupees (Rs.) and not in any other currency. Each line of the generated summary should discuss comparative details of a particular product with other products. The summary should not have any redundant information among different lines and should only pertain to the current three recommended products for the query. Strictly write the comparative summary in the following format: {'Comparative Summary': concise 225-250 word comparative summary in one paragraph.}"

# Initialize a dictionary to store the comparative summaries for each query
output_dict = {}

# Iterate over the products in the data
for i in range(0, len(product_data), 3):
    # Extract relevant fields for each product
    query = product_data[i]['query']
    product_titles = [product_data[i+j]['product_title'] for j in range(3)]
    base_prices = [product_data[i+j]['base_price'] for j in range(3)]
    final_prices = [product_data[i+j]['final_price'] for j in range(3)]
    opinion_summaries = [product_data[i+j]['Product_Opinion_Summary'] for j in range(3)]
    
    # Define a template for the input prompt
    input_template = f"""
    {system_msg}

    {prompt}

    User Query: {query}

    Product I:
    Title: {product_titles[0]}
    Base Price: Rs. {base_prices[0]}
    Final Price: Rs. {final_prices[0]}
    Opinion Summary: {opinion_summaries[0]}

    Product II:
    Title: {product_titles[1]}
    Base Price: Rs. {base_prices[1]}
    Final Price: Rs. {final_prices[1]}
    Opinion Summary: {opinion_summaries[1]}

    Product III:
    Title: {product_titles[2]}
    Base Price: Rs. {base_prices[2]}
    Final Price: Rs. {final_prices[2]}
    Opinion Summary: {opinion_summaries[2]}

    Instructions: 
    Please generate the comparative summary following the above instructions in one single paragraph of 225-250 words.
    """

    # Generate the comparative summary
    with torch.cuda.device(device):
        output = model.generate(
            max_length=4500,  # Increased max_length
            num_return_sequences=1,
            do_sample=True,
            top_k=25,
            top_p=0.95,
            num_beams=3,
            early_stopping=True,
            temperature=0.7,
            input_ids=tokenizer(input_template, return_tensors="pt").input_ids.to(device)
        )

    # Decode the comparative summary
    comparative_summary = tokenizer.decode(output[0], skip_special_tokens=True)

    # Add the query and comparative summary to the dictionary
    output_dict[query] = {'Comparative Summary': comparative_summary}

    # Print the query and comparative summary
    print("Query:", query)
    print("Comparative Summary:", comparative_summary)
    print("\n")  # Print newline for better readability between summaries

Using CUDA device: NVIDIA A100 80GB PCIe


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Query: Vivo 5G mobiles with curved display
Comparative Summary: 
    You are an expert in generating summaries that compare and justify the order of three recommended products on an e-commerce platform to help customers make purchase decisions. You carefully follow every instruction to answer faithfully, truthfully, and accurately in the specified format.

    ### Instruction: Write a concise single paragraph summary that explains the order in which the three recommended products are presented for a given user query. Use information such as query, base price, final price, and Product Opinion Summary to justify the ranking. The comparative summary should provide a clear rationale for why product I is ranked above product II and why product II is ranked above product III, without providing individual explanations for each product. Write in a clear, engaging style for a general audience, avoiding overly technical language or jargon. Aim for a conversational yet professional tone while bei

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Query: best 5G mobile with good camera and high performance under 25000
Comparative Summary: 
    You are an expert in generating summaries that compare and justify the order of three recommended products on an e-commerce platform to help customers make purchase decisions. You carefully follow every instruction to answer faithfully, truthfully, and accurately in the specified format.

    ### Instruction: Write a concise single paragraph summary that explains the order in which the three recommended products are presented for a given user query. Use information such as query, base price, final price, and Product Opinion Summary to justify the ranking. The comparative summary should provide a clear rationale for why product I is ranked above product II and why product II is ranked above product III, without providing individual explanations for each product. Write in a clear, engaging style for a general audience, avoiding overly technical language or jargon. Aim for a conversational ye

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Query: buy dhop
Comparative Summary: 
    You are an expert in generating summaries that compare and justify the order of three recommended products on an e-commerce platform to help customers make purchase decisions. You carefully follow every instruction to answer faithfully, truthfully, and accurately in the specified format.

    ### Instruction: Write a concise single paragraph summary that explains the order in which the three recommended products are presented for a given user query. Use information such as query, base price, final price, and Product Opinion Summary to justify the ranking. The comparative summary should provide a clear rationale for why product I is ranked above product II and why product II is ranked above product III, without providing individual explanations for each product. Write in a clear, engaging style for a general audience, avoiding overly technical language or jargon. Aim for a conversational yet professional tone while being fluent and coherent. The

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Query: best 5G mobile under 15000 with good camera and great performance from Realme
Comparative Summary: 
    You are an expert in generating summaries that compare and justify the order of three recommended products on an e-commerce platform to help customers make purchase decisions. You carefully follow every instruction to answer faithfully, truthfully, and accurately in the specified format.

    ### Instruction: Write a concise single paragraph summary that explains the order in which the three recommended products are presented for a given user query. Use information such as query, base price, final price, and Product Opinion Summary to justify the ranking. The comparative summary should provide a clear rationale for why product I is ranked above product II and why product II is ranked above product III, without providing individual explanations for each product. Write in a clear, engaging style for a general audience, avoiding overly technical language or jargon. Aim for a conv

# new BEST but gives "long and technical"

In [13]:
import json
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Check if CUDA is available
if torch.cuda.is_available():
    # Specify CUDA device
    device = torch.device("cuda:1")
    print(f"Using CUDA device: {torch.cuda.get_device_name(device)}")
else:
    device = torch.device("cpu")
    print("CUDA is not available. Using CPU.")

# Load the Mistral 7B model
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
tokenizer.pad_token_id = tokenizer.eos_token_id
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2").to(device)

# Load the product data from finalc10.json
with open('finalc10.json', 'r') as f:
    product_data = json.load(f)

# System message and prompt
system_msg = "You are an expert in generating summaries that provides comparison beteween the three recommended products on an e-commerce platform to help customers make purchase decisions. You carefully follow every instruction to answer faithfully, truthfully, and accurately in the specified format."

prompt = "### Instruction: Write a balanced and concise single-  paragraph summary strictly in 225- 250 words that highlight the comparisons of the three recommended products that are presented to a user for the query the user has entered. Use information such as query,  base price, final price, and Product Opinion Summary to justify the comparisons between all three products. The summary should provide a clear rationale for why Product I is better than Product II and Product III and why Product II is better than Product III. Write in a clear, engaging style for a general audience, avoiding overly technical language or jargon. Aim for a conversational yet professional tone while being fluent and coherent. The summary should correctly and faithfully capture the key differences and justify the reasons for the comparisons among the three products you made. Ensure that prices are mentioned in Indian Rupees (Rs.) and not in any other currency. Each line of the generated summary should compare a product with other products. The summary should not have redundant and highly technical information among different lines and should only pertain to the current three recommended products for the query. Moreover, the summary should not repeat the comparison information again and again. Strictly write the summary in the following format: {'Summary': concise, balanced 225-250 word summary in one paragraph.}"
# Initialize a dictionary to store the comparative explanations for each query

output_dict = {}

# Iterate over the products in the data
for i in range(0, len(product_data), 3):
    # Extract relevant fields for each product
    query = product_data[i]['query']
    product_titles = [product_data[i+j]['product_title'] for j in range(3)]
    base_prices = [product_data[i+j]['base_price'] for j in range(3)]
    final_prices = [product_data[i+j]['final_price'] for j in range(3)]
    opinion_summaries = [product_data[i+j]['Product_Opinion_Summary'] for j in range(3)]
    # Define a template for the input prompt
    input_template = f"""
    {system_msg}

    {prompt}

    User Query: {query}

    Product I:
    Title: {product_titles[0]}
    Base Price: Rs. {base_prices[0]}
    Final Price: Rs. {final_prices[0]}
    Opinion Summary: {opinion_summaries[0]}

    Product II:
    Title: {product_titles[1]}
    Base Price: Rs. {base_prices[1]}
    Final Price: Rs. {final_prices[1]}
    Opinion Summary: {opinion_summaries[1]}

    Product III:
    Title: {product_titles[2]}
    Base Price: Rs. {base_prices[2]}
    Final Price: Rs. {final_prices[2]}
    Opinion Summary: {opinion_summaries[2]}

    Instructions: 
    
    Please generate the comparative explanation following the above instructions in one single paragraph of 225-250 words.
    """

    # Generate the comparative explanation
    with torch.cuda.device(device):
        output = model.generate(
            max_length=4500,  # Increased max_length
            num_return_sequences=1,
            do_sample=True,
            top_k=25,
            top_p=0.95,
            num_beams=3,
            early_stopping=True,
            temperature=0.7,
            input_ids=tokenizer(input_template, return_tensors="pt").input_ids.to(device)
        )

    # Decode the comparative explanation
    comparative_explanation = tokenizer.decode(output[0], skip_special_tokens=True)

    # Add the query and comparative explanation to the dictionary
    output_dict[query] = {'Comparative Explanation': comparative_explanation}

    # Print the query and comparative explanation
    print("Query:", query)
    print("Comparative Explanation:", comparative_explanation)
    print("\n")  # Print newline for better readability between explanations


Using CUDA device: NVIDIA A100 80GB PCIe


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Query: Vivo 5G mobiles with curved display
Comparative Explanation: 
    You are an expert in generating summaries that provides comparison beteween the three recommended products on an e-commerce platform to help customers make purchase decisions. You carefully follow every instruction to answer faithfully, truthfully, and accurately in the specified format.

    ### Instruction: Write a balanced and concise single-  paragraph summary strictly in 225- 250 words that highlight the comparisons of the three recommended products that are presented to a user for the query the user has entered. Use information such as query,  base price, final price, and Product Opinion Summary to justify the comparisons between all three products. The summary should provide a clear rationale for why Product I is better than Product II and Product III and why Product II is better than Product III. Write in a clear, engaging style for a general audience, avoiding overly technical language or jargon. Aim for 

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Query: best 5G mobile with good camera and high performance under 25000
Comparative Explanation: 
    You are an expert in generating summaries that provides comparison beteween the three recommended products on an e-commerce platform to help customers make purchase decisions. You carefully follow every instruction to answer faithfully, truthfully, and accurately in the specified format.

    ### Instruction: Write a balanced and concise single-  paragraph summary strictly in 225- 250 words that highlight the comparisons of the three recommended products that are presented to a user for the query the user has entered. Use information such as query,  base price, final price, and Product Opinion Summary to justify the comparisons between all three products. The summary should provide a clear rationale for why Product I is better than Product II and Product III and why Product II is better than Product III. Write in a clear, engaging style for a general audience, avoiding overly technical

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Query: buy dhop
Comparative Explanation: 
    You are an expert in generating summaries that provides comparison beteween the three recommended products on an e-commerce platform to help customers make purchase decisions. You carefully follow every instruction to answer faithfully, truthfully, and accurately in the specified format.

    ### Instruction: Write a balanced and concise single-  paragraph summary strictly in 225- 250 words that highlight the comparisons of the three recommended products that are presented to a user for the query the user has entered. Use information such as query,  base price, final price, and Product Opinion Summary to justify the comparisons between all three products. The summary should provide a clear rationale for why Product I is better than Product II and Product III and why Product II is better than Product III. Write in a clear, engaging style for a general audience, avoiding overly technical language or jargon. Aim for a conversational yet profes

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Query: best 5G mobile under 15000 with good camera and great performance from Realme
Comparative Explanation: 
    You are an expert in generating summaries that provides comparison beteween the three recommended products on an e-commerce platform to help customers make purchase decisions. You carefully follow every instruction to answer faithfully, truthfully, and accurately in the specified format.

    ### Instruction: Write a balanced and concise single-  paragraph summary strictly in 225- 250 words that highlight the comparisons of the three recommended products that are presented to a user for the query the user has entered. Use information such as query,  base price, final price, and Product Opinion Summary to justify the comparisons between all three products. The summary should provide a clear rationale for why Product I is better than Product II and Product III and why Product II is better than Product III. Write in a clear, engaging style for a general audience, avoiding ove

# 🌲 new BEST but gives "little small and less technical" 125-150 words

# Model Saving

In [1]:
import json
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Check if CUDA is available
if torch.cuda.is_available():
    # Specify CUDA device
    device = torch.device("cuda:1")
    print(f"Using CUDA device: {torch.cuda.get_device_name(device)}")
else:
    device = torch.device("cpu")
    print("CUDA is not available. Using CPU.")

# Load the Mistral 7B model
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
tokenizer.pad_token_id = tokenizer.eos_token_id
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2").to(device)

# Load the product data from finalc10.json
with open('finalc10.json', 'r') as f:
    product_data = json.load(f)

system_msg = "You are an expert in generating summaries that provides comparison beteween the three recommended products on an e-commerce platform to help customers make purchase decisions. Follow everything written in the below prompt carefully and truthfully and stick to the word limit mentioned in the prompt. Also carefully follow every instruction to answer faithfully, truthfully, and accurately in the specified format."

prompt = "### Instruction: Write a balanced and concise single-paragraph summary strictly in 125- 150 word limit that highlight the comparisons of the three recommended products that are presented to a user for the query the user has entered. Use information such as query,  base price, final price, and Product Opinion Summary to justify the comparisons between all three products. The summary should provide a clear rationale for why Product I is better than Product II and Product III and why Product II is better than Product III. Write in a clear, engaging style for a general audience, avoiding overly technical language or jargon. Aim for a conversational yet professional tone while being fluent and coherent. The summary should correctly and faithfully capture the key differences and justify the reasons for the comparisons among the three products you made. Ensure that prices are mentioned in Indian Rupees (Rs.) and not in any other currency. Each line of the generated summary should compare a product with other products. The summary should not have redundant and highly technical information among different lines and should only pertain to the current three recommended products for the query. Moreover, the summary should not repeat the comparison information again and again. Strictly write the summary in the following format: {'Summary': concise, balanced 125-150 word summary in one paragraph.}"
output_dict = {}

# Iterate over the products in the data
for i in range(0, len(product_data), 3):
    # Extract relevant fields for each product
    query = product_data[i]['query']
    product_titles = [product_data[i+j]['product_title'] for j in range(3)]
    base_prices = [product_data[i+j]['base_price'] for j in range(3)]
    final_prices = [product_data[i+j]['final_price'] for j in range(3)]
    opinion_summaries = [product_data[i+j]['Product_Opinion_Summary'] for j in range(3)]
    # Define a template for the input prompt
    input_template = f"""
    {system_msg}

    {prompt}

    User Query: {query}

    Product I:
    Title: {product_titles[0]}
    Base Price: Rs. {base_prices[0]}
    Final Price: Rs. {final_prices[0]}
    Opinion Summary: {opinion_summaries[0]}

    Product II:
    Title: {product_titles[1]}
    Base Price: Rs. {base_prices[1]}
    Final Price: Rs. {final_prices[1]}
    Opinion Summary: {opinion_summaries[1]}

    Product III:
    Title: {product_titles[2]}
    Base Price: Rs. {base_prices[2]}
    Final Price: Rs. {final_prices[2]}
    Opinion Summary: {opinion_summaries[2]}

    Instructions: 
    
    Follow everything written in the prompt carefully and truthfully and stick to the word limit mentioned in the prompt.
    """

    # Generate the comparative explanation
    with torch.cuda.device(device):
        output = model.generate(
            max_length=4500,  # Increased max_length
            num_return_sequences=1,
            do_sample=True,
            top_k=25,
            top_p=0.95,
            num_beams=3,
            early_stopping=True,
            temperature=0.7,
            input_ids=tokenizer(input_template, return_tensors="pt").input_ids.to(device)
        )

    # Decode the comparative explanation
    comparative_explanation = tokenizer.decode(output[0], skip_special_tokens=True)

    # Add the query and comparative explanation to the dictionary
    output_dict[query] = {'Comparative Explanation': comparative_explanation}

    # Print the query and comparative explanation
    print("Query:", query)
    print("Comparative Explanation:", comparative_explanation)
    print("\n")  # Print newline for better readability between explanations


Using CUDA device: NVIDIA A100 80GB PCIe


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Query: Vivo 5G mobiles with curved display
Comparative Explanation: 
    You are an expert in generating summaries that provides comparison beteween the three recommended products on an e-commerce platform to help customers make purchase decisions. Follow everything written in the below prompt carefully and truthfully and stick to the word limit mentioned in the prompt. Also carefully follow every instruction to answer faithfully, truthfully, and accurately in the specified format.

    ### Instruction: Write a balanced and concise single-paragraph summary strictly in 125- 150 word limit that highlight the comparisons of the three recommended products that are presented to a user for the query the user has entered. Use information such as query,  base price, final price, and Product Opinion Summary to justify the comparisons between all three products. The summary should provide a clear rationale for why Product I is better than Product II and Product III and why Product II is better t

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Query: best 5G mobile with good camera and high performance under 25000
Comparative Explanation: 
    You are an expert in generating summaries that provides comparison beteween the three recommended products on an e-commerce platform to help customers make purchase decisions. Follow everything written in the below prompt carefully and truthfully and stick to the word limit mentioned in the prompt. Also carefully follow every instruction to answer faithfully, truthfully, and accurately in the specified format.

    ### Instruction: Write a balanced and concise single-paragraph summary strictly in 125- 150 word limit that highlight the comparisons of the three recommended products that are presented to a user for the query the user has entered. Use information such as query,  base price, final price, and Product Opinion Summary to justify the comparisons between all three products. The summary should provide a clear rationale for why Product I is better than Product II and Product III a

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Query: buy dhop
Comparative Explanation: 
    You are an expert in generating summaries that provides comparison beteween the three recommended products on an e-commerce platform to help customers make purchase decisions. Follow everything written in the below prompt carefully and truthfully and stick to the word limit mentioned in the prompt. Also carefully follow every instruction to answer faithfully, truthfully, and accurately in the specified format.

    ### Instruction: Write a balanced and concise single-paragraph summary strictly in 125- 150 word limit that highlight the comparisons of the three recommended products that are presented to a user for the query the user has entered. Use information such as query,  base price, final price, and Product Opinion Summary to justify the comparisons between all three products. The summary should provide a clear rationale for why Product I is better than Product II and Product III and why Product II is better than Product III. Write in a

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Query: best 5G mobile under 15000 with good camera and great performance from Realme
Comparative Explanation: 
    You are an expert in generating summaries that provides comparison beteween the three recommended products on an e-commerce platform to help customers make purchase decisions. Follow everything written in the below prompt carefully and truthfully and stick to the word limit mentioned in the prompt. Also carefully follow every instruction to answer faithfully, truthfully, and accurately in the specified format.

    ### Instruction: Write a balanced and concise single-paragraph summary strictly in 125- 150 word limit that highlight the comparisons of the three recommended products that are presented to a user for the query the user has entered. Use information such as query,  base price, final price, and Product Opinion Summary to justify the comparisons between all three products. The summary should provide a clear rationale for why Product I is better than Product II and 

# feature extract

In [14]:
import json
import spacy

nlp = spacy.load("en_core_web_sm")

def extract_features(text):
    doc = nlp(text)
    return [chunk.text.lower() for chunk in doc.noun_chunks]  # Convert to lowercase to standardize features

def process_json_file(input_filename, output_filename):
    with open(input_filename, 'r') as file:
        data = json.load(file)
    
    results = []
    
    for i in range(0, len(data), 3):
        query = data[i]['query']
        features_list = []  # List to hold feature sets from each product

        for j in range(3):
            product = data[i + j]
            # Extract noun phrases from Product Opinion Summary and add product title
            features = extract_features(product['Product_Opinion_Summary']) + [product['product_title'].lower()]
            features.extend([str(product['base_price']), str(product['final_price'])])  # Adding prices as features
            features_set = set(features)
            features_list.append(features_set)
        
        # Perform union of all three sets to get combined features
        combined_features = set.union(*features_list)
        
        results.append({
            "query": query,
            "features": list(combined_features)
        })
    
    with open(output_filename, 'w') as outfile:
        json.dump(results, outfile, indent=4)

# Usage
input_filename = 'finalc10.json'
output_filename = 'features.json'
process_json_file(input_filename, output_filename)


# cex.json structure creation

In [15]:
import json

# Path to the input JSON file
input_file_path = 'finalc10.json'
# Path to the output JSON file
output_file_path = 'cex.json'

def process_file(input_path, output_path):
    # Load the data from the input JSON file
    with open(input_path, 'r') as file:
        data = json.load(file)

    # Dictionary to hold the processed data with unique queries
    processed_data = {}

    # Extract required information from each product entry
    for entry in data:
        query = entry["query"]
        # If the query is already processed, skip to the next
        if query not in processed_data:
            processed_data[query] = {
                "query": query,
                "comparative_explanation_summary": ""
            }

    # Convert the processed data back to a list format
    final_data = list(processed_data.values())

    # Write the processed data to the output JSON file
    with open(output_path, 'w') as file:
        json.dump(final_data, file, indent=4)

# Call the function to process the file
process_file(input_file_path, output_file_path)


# Normalize the cex.json containg comaparative summary

In [4]:
import json

def normalize_text(text):
    """Convert text to lowercase to normalize it."""
    return text.lower()

def normalize_summaries(input_filename, output_filename):
    # Load the JSON data from the file
    with open(input_filename, 'r') as file:
        data = json.load(file)
    
    # Iterate through each item and normalize the comparative_explanation_summary
    for item in data:
        if 'comparative_explanation_summary' in item:
            item['comparative_explanation_summary'] = normalize_text(item['comparative_explanation_summary'])
    
    # Write the normalized data to a new JSON file
    with open(output_filename, 'w') as outfile:
        json.dump(data, outfile, indent=4, ensure_ascii=False)

# Usage
input_filename = 'cex_lesswords.json'  # Make sure to update this if your input file has a different name
output_filename = 'normal.json'
normalize_summaries(input_filename, output_filename)


# Feature Matching Ratio (FMR).

In [18]:
import json
import csv
from collections import Counter

def load_json_data(filename):
    """ Load data from a JSON file """
    with open(filename, 'r') as file:
        data = json.load(file)
    return data

def compute_fmr(features, summary):
    """ Compute Feature Matching Ratio (FMR) """
    summary_tokens = summary.split()  # Split summary into tokens
    summary_counter = Counter(summary_tokens)  # Count occurrence of each token in the summary
    feature_set = set(features)  # Convert features list to set for faster operations

    matched_features = [word for word in summary_counter if word in feature_set]
    total_features_in_summary = sum(summary_counter.values())

    if total_features_in_summary > 0:
        fmr_score = len(matched_features) / total_features_in_summary
    else:
        fmr_score = 0  # To handle case with no features in summary

    return round(fmr_score, 3)  # Round to two decimal places

def evaluate_fmr(features_file, summaries_file, output_file):
    # Load features and summaries
    features_data = load_json_data(features_file)
    summaries_data = load_json_data(summaries_file)

    # Prepare for output
    results = []
    total_fmr = 0  # Sum of all FMR scores for average calculation
    count_fmr = 0  # Count of FMR scores included in average

    # Ensure we have features for each query
    features_dict = {item['query']: item['features'] for item in features_data}

    # Calculate FMR for each summary
    for summary in summaries_data:
        query = summary['query']
        if query in features_dict:
            fmr_score = compute_fmr(features_dict[query], summary['comparative_explanation_summary'])
            results.append({'query': query, 'FMR score': fmr_score})
            total_fmr += fmr_score
            count_fmr += 1
        else:
            results.append({'query': query, 'FMR score': 'No features available'})

    # Calculate average FMR
    if count_fmr > 0:
        average_fmr = round(total_fmr / count_fmr, 3)
    else:
        average_fmr = 0

    # Write results to a CSV file
    with open(output_file, 'w', newline='') as csvfile:
        fieldnames = ['query', 'FMR score']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(results)
        # Write average FMR at the end of the file
        writer.writerow({'query': 'Average FMR', 'FMR score': average_fmr})

# Usage
features_file = 'features.json'
summaries_file = 'normal_cex_long_tech.json'
output_file = 'fmr.csv'
evaluate_fmr(features_file, summaries_file, output_file)


# Feature Coverage Ratio (FCR)

In [19]:
import json
import csv
from collections import Counter

def load_json_data(filename):
    """ Load data from a JSON file """
    with open(filename, 'r') as file:
        data = json.load(file)
    return data

def compute_fcr(features, summary):
    """ Compute Feature Coverage Ratio (FCR) """
    summary_tokens = set(summary.split())  # Split summary into tokens and make a set to remove duplicates
    feature_set = set(features)  # Convert features list to set for faster operations

    matched_features = feature_set.intersection(summary_tokens)
    if len(feature_set) > 0:
        fcr_score = len(matched_features) / len(feature_set)
    else:
        fcr_score = 0  # Handle case where no features are defined

    return round(fcr_score, 3)  # Round to three decimal places

def evaluate_fcr(features_file, summaries_file, output_file):
    # Load features and summaries
    features_data = load_json_data(features_file)
    summaries_data = load_json_data(summaries_file)

    # Prepare for output
    results = []
    total_fcr = 0  # Sum of all FCR scores for average calculation
    count_fcr = 0  # Count of FCR scores included in average

    # Ensure we have features for each query
    features_dict = {item['query']: item['features'] for item in features_data}

    # Calculate FCR for each summary
    for summary in summaries_data:
        query = summary['query']
        if query in features_dict:
            fcr_score = compute_fcr(features_dict[query], summary['comparative_explanation_summary'])
            results.append({'query': query, 'FCR score': fcr_score})
            total_fcr += fcr_score
            count_fcr += 1
        else:
            results.append({'query': query, 'FCR score': 'No features available'})

    # Calculate average FCR
    if count_fcr > 0:
        average_fcr = round(total_fcr / count_fcr, 3)
    else:
        average_fcr = 0

    # Write results to a CSV file
    with open(output_file, 'w', newline='') as csvfile:
        fieldnames = ['query', 'FCR score']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(results)
        # Write average FCR at the end of the file
        writer.writerow({'query': 'Average FCR', 'FCR score': average_fcr})

# Usage
features_file = 'features.json'
summaries_file = 'normal_cex_long_tech.json'
output_file = 'fcr.csv'
evaluate_fcr(features_file, summaries_file, output_file)


# Feature Diversity (DIV)

In [20]:
import json
import csv
from collections import Counter

def load_json_data(filename):
    """ Load data from a JSON file """
    with open(filename, 'r') as file:
        data = json.load(file)
    return data

def compute_div(features, summary):
    """ Compute Feature Diversity (DIV) """
    summary_tokens = set(summary.split())  # Split summary into tokens and make a set to remove duplicates
    feature_set = set(features)  # Convert features list to set for faster operations

    matched_features = feature_set.intersection(summary_tokens)
    distinct_features_in_summary = len(matched_features)

    if len(feature_set) > 0:
        div_score = distinct_features_in_summary / len(feature_set)
    else:
        div_score = 0  # Handle case where no features are defined

    return round(div_score, 3)  # Round to three decimal places

def evaluate_div(features_file, summaries_file, output_file):
    # Load features and summaries
    features_data = load_json_data(features_file)
    summaries_data = load_json_data(summaries_file)

    # Prepare for output
    results = []
    total_div = 0  # Sum of all DIV scores for average calculation
    count_div = 0  # Count of DIV scores included in average

    # Ensure we have features for each query
    features_dict = {item['query']: item['features'] for item in features_data}

    # Calculate DIV for each summary
    for summary in summaries_data:
        query = summary['query']
        if query in features_dict:
            div_score = compute_div(features_dict[query], summary['comparative_explanation_summary'])
            results.append({'query': query, 'DIV score': div_score})
            total_div += div_score
            count_div += 1
        else:
            results.append({'query': query, 'DIV score': 'No features available'})

    # Calculate average DIV
    if count_div > 0:
        average_div = round(total_div / count_div, 3)
    else:
        average_div = 0

    # Write results to a CSV file
    with open(output_file, 'w', newline='') as csvfile:
        fieldnames = ['query', 'DIV score']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(results)
        # Write average DIV at the end of the file
        writer.writerow({'query': 'Average DIV', 'DIV score': average_div})

# Usage
features_file = 'features.json'
summaries_file = 'normal_cex_long_tech.json'
output_file = 'div.csv'
evaluate_div(features_file, summaries_file, output_file)


# Distinct-1 and Distinct-2 (D-1, D-2)

In [14]:
import json
import csv
from nltk import bigrams, word_tokenize
from collections import Counter

def load_json_data(filename):
    """ Load data from a JSON file """
    with open(filename, 'r') as file:
        data = json.load(file)
    return data

def calculate_distinct_n(tokens, n=1):
    """ Calculate distinct-n metrics for tokens """
    if n == 1:
        n_grams = tokens
    else:
        n_grams = list(bigrams(tokens))

    total_n_grams = len(n_grams)
    unique_n_grams = len(set(n_grams))

    return round(unique_n_grams / total_n_grams if total_n_grams > 0 else 0, 3)

def evaluate_distinct_metrics(summaries_file, output_file):
    summaries_data = load_json_data(summaries_file)
    results = []
    total_d1 = 0
    total_d2 = 0
    count = 0

    for summary in summaries_data:
        tokens = word_tokenize(summary['comparative_explanation_summary'].lower())
        d1_score = calculate_distinct_n(tokens, 1)
        d2_score = calculate_distinct_n(tokens, 2)
        results.append({
            'query': summary['query'],
            'D1 score': d1_score,
            'D2 score': d2_score
        })
        total_d1 += d1_score
        total_d2 += d2_score
        count += 1

    average_d1 = round(total_d1 / count, 3) if count > 0 else 0
    average_d2 = round(total_d2 / count, 3) if count > 0 else 0

    # Write results to a CSV file
    with open(output_file, 'w', newline='') as csvfile:
        fieldnames = ['query', 'D1 score', 'D2 score']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(results)
        writer.writerow({'query': 'Average', 'D1 score': average_d1, 'D2 score': average_d2})

# Usage
summaries_file = 'normal_cex_lesswords.json'
output_file = 'distinct.csv'
evaluate_distinct_metrics(summaries_file, output_file)


# Flesch Reading Ease

In [16]:
pip install textstat


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 kB 96.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 86.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [21]:
import json
import csv
import textstat

def load_json_data(filename):
    """Load data from a JSON file."""
    with open(filename, 'r') as file:
        data = json.load(file)
    return data

def evaluate_readability(summaries_file, output_file):
    summaries_data = load_json_data(summaries_file)
    results = []
    total_flesh_score = 0
    count = 0

    for summary in summaries_data:
        flesh_score = textstat.flesch_reading_ease(summary['comparative_explanation_summary'])
        results.append({
            'query': summary['query'],
            'Flesh score': round(flesh_score, 3)
        })
        total_flesh_score += flesh_score
        count += 1

    average_flesh_score = round(total_flesh_score / count, 3) if count > 0 else 0

    # Write results to a CSV file
    with open(output_file, 'w', newline='') as csvfile:
        fieldnames = ['query', 'Flesh score']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(results)
        # Write average Flesch score at the end of the file
        writer.writerow({'query': 'Average', 'Flesh score': average_flesh_score})

# Usage
summaries_file = 'normal_cex_long_tech.json'
output_file = 'flesch.csv'
evaluate_readability(summaries_file, output_file)
